# Import libraries

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0,'..')
import glob 
#!pip install openpyxl

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import h5py
import re

from functools import partial
from multiprocessing import Pool

from lhcsmapi.Time import Time
from lhcsmapi.metadata import signal_metadata
from lhcsmapi.pyedsl.dbsignal.post_mortem.PmDbRequest import PmDbRequest
from lhcsmapi.analysis.RbCircuitQuery import RbCircuitQuery

from src.acquisitions.current_voltage_diode_leads_nxcals import CurrentVoltageDiodeLeadsNXCALS
from src.acquisitions.current_voltage_diode_leads_pm import CurrentVoltageDiodeLeadsPM
from src.acquisitions.ee_t_res_pm import EETResPM
from src.acquisitions.ee_u_dump_res_pm import EEUDumpResPM
from src.acquisitions.leads import Leads
from src.acquisitions.pc_pm import PCPM
from src.acquisitions.qh_pm import QHPM
from src.acquisitions.voltage_logic_iqps import VoltageLogicIQPS
from src.acquisitions.voltage_nqps import VoltageNQPS
from src.acquisitions.voltage_nxcals import VoltageNXCALS

from src.utils.utils import log_acquisition
from src.utils.hdf_tools import acquisition_to_hdf5, load_from_hdf_with_regex
from src.utils.mp3_excel_processing import get_fgc_timestamp, get_fgc_timestamp_missing
from src.visualisation.visualisation import plot_hdf

# Select events to download

In [ ]:
mp3_fpa_df = pd.read_csv("../data/RB_TC_extract_2021_11_22_processed.csv")
mp3_fpa_df_unique = mp3_fpa_df.drop_duplicates(subset=['timestamp_fgc', 'Circuit Name'])

In [ ]:
lower_limit = Time.to_unix_timestamp('2021-01-01 00:00:00+01:00')
mp3_fpa_df_period = mp3_fpa_df_unique[mp3_fpa_df_unique['timestamp_fgc'] >= lower_limit].reset_index(drop=True) 
len(mp3_fpa_df_period)

In [ ]:
signal_groups = [PCPM, VoltageNXCALS, VoltageNQPS, VoltageLogicIQPS, EEUDumpResPM, QHPM]
file_dir = Path('/eos/project/m/ml-for-alarm-system/private/RB_signals')

## Test query

In [ ]:
# quench with precursor
fpa_identifier = {'circuit_type': 'RB',
                 'circuit_name': 'RB.A78',
                 'timestamp_fgc': 1616962174400000000}

In [ ]:
context_path = file_dir / "context_test1"

for signal_group in signal_groups:
    group = signal_group(**fpa_identifier, spark=spark)
    acquisition_to_hdf5(acquisition=group, 
                        file_dir=file_dir, 
                        context_dir_name="context_test1",
                        failed_queries_dir_name="failed_test1",
                        data_dir_name="data_test1")
        
log_acquisition(identifier=fpa_identifier, log_data={"download_complete": True}, log_path=context_path)

## Load data

In [ ]:
file_name = f"{fpa_identifier['circuit_type']}_{fpa_identifier['circuit_name']}_{fpa_identifier['timestamp_fgc']}.hdf5"
file_path = file_dir / Path('data_test1') / file_name

signals = ['I_MEAS','VoltageNQPS.*U_DIODE','VoltageNXCALS.*U_DIODE','I_EARTH_PCNT', 'IEARTH.I_EARTH','U_QS0','U_1','U_2','I_HDS', 'U_HDS','EEUDumpResPM']
len(signals)

In [ ]:
data = load_from_hdf_with_regex(file_path)

In [ ]:
plot_hdf(data, column_regex=signals)

## Query all FGC events

In [ ]:
# Time.to_string(1514761200000000000), Time.to_unix_timestamp('2018-01-01 00:00:00+01:00')
mp3_fpa_df_period = mp3_fpa_df_unique[mp3_fpa_df_unique['timestamp_fgc'] >= 1609455600000000000].reset_index(drop=True) 
len(mp3_fpa_df_period)

signals = ['I_MEAS','VoltageNQPS.*U_DIODE','VoltageNXCALS.*U_DIODE','I_EARTH_PCNT', 'IEARTH.I_EARTH','U_QS0','U_1','U_2','I_HDS', 'U_HDS','EEUDumpResPM']

In [ ]:
for index, row in mp3_fpa_df_period.iterrows():
    
    fpa_identifier = {'circuit_type': row['Circuit Family'],
                      'circuit_name': row['Circuit Name'],
                      'timestamp_fgc': int(row['timestamp_fgc'])}
    file_name = f"{fpa_identifier['circuit_type']}_{fpa_identifier['circuit_name']}_{fpa_identifier['timestamp_fgc']}.hdf5"
    Path(file_dir / Path('20220707_plots')).mkdir(parents=True, exist_ok=True)
    
    if not os.path.isfile(str(file_dir / Path('20220707_plots') / f"{fpa_identifier['circuit_type']}_{fpa_identifier['circuit_name']}_{fpa_identifier['timestamp_fgc']}.png")):
        for signal_group in signal_groups:
            group = signal_group(**fpa_identifier, spark=spark)
            acquisition_to_hdf5(acquisition=group, 
                                file_dir=file_dir,
                                context_dir_name="20220707_context",
                                failed_queries_dir_name="20220707_failed",
                                data_dir_name="20220707_data")

        #log_acquisition(identifier=fpa_identifier, log_data={"download_complete": True}, log_path=context_path)

        file_path = file_dir / Path('20220707_data') / file_name
        data = load_from_hdf_with_regex(file_path)
        fig = plt.figure(figsize=(20, 15))
        for i, s in enumerate(signals):
            fig.add_subplot(4, 3, i+1)
            n_signals = 0
            for df in data:
                if bool(re.search(s, df.columns.values[0])):
                    plt.plot(df.index.values, df.values)
                    n_signals += 1
            plt.title(f"{s} ({n_signals})")

        plt.savefig(str(file_dir / Path('20220707_plots') / f"{fpa_identifier['circuit_type']}_{fpa_identifier['circuit_name']}_{fpa_identifier['timestamp_fgc']}.png"))
        plt.close()
    else:
        print(f"done: {file_name}")